# Building LLM-based applications for writting discharge summary

In this notebook, we will build an application to writting discharge summary.

For required packages, please run:

```bash
poetry install
```

There are a few methods explored in this notebook:
1. Stuff with human and system prompt
2. Stuff with human prompt
3. Decompose method
4. MapReduce method
5. Refine method

In [ ]:
import os
import sys
from pathlib import Path

import nest_asyncio
from dotenv import load_dotenv
from IPython.display import Markdown
from langchain_openai import AzureChatOpenAI

nest_asyncio.apply()

load_dotenv()
llm = AzureChatOpenAI(
    deployment_name="gpt_35_16k",
    model_name="gpt-35-turbo-16k",
    temperature=0,
)

# Add notebooks parent directory to path
project_folder = Path(os.getcwd()).parent
sys.path.append(str(project_folder))
output_folder = project_folder / "data" / "output"
case_dir = project_folder / "data" / 'processed'
if not case_dir.exists():
    raise FileNotFoundError(f"Cases directory not found at {case_dir}")
if not output_folder.exists():
    output_folder.mkdir(parents=True)

In [ ]:
# Local files
from helpers import read_json_file, read_markdown_file

# Read cases

In [ ]:
# Filter those that are _original.md - ignore other languages for now
file_list = [
    f for f in (case_dir / 'merged').glob('*.md')
    if f.stem.endswith('_original') and f.stem.find('Orthopaedic') != -1
]
work_with_file = file_list[0]

case = read_markdown_file(path=work_with_file)

# Print first 1000 characters of the case
Markdown(case[:1000] + '...')

In [ ]:
json_file_list = [f for f in (case_dir / 'markdown').glob('*.json')
                  if f.stem.endswith('_original') and
                     f.stem.find('Orthopaedic') != -1]

# Read and parse the json file as a dictionary
json_file = json_file_list[0]

case_object = read_json_file(json_file)
str(case_object)

# Basic method (Naive)

In [ ]:
from basic.basic import basic_chain

out = basic_chain(llm=llm).single.invoke({"notes": case})
Markdown(out)

In [ ]:
from basic.basic import create_multiple_basic_prompts

stuff_df = create_multiple_basic_prompts(case=case, n=2, llm=llm)
stuff_df.to_csv(output_folder / "stuff_df_final.csv")

# Decompose Technique

In [ ]:
from decompose import combine_all_sections

decompose_df = combine_all_sections(case=case_object, llm=llm, n = 1)
decompose_df.to_csv(output_folder / "decompose_df_final.csv")

# Map Reduce Prompt Techniques

In [ ]:
from map_reduce import create_mapreduce_df

mapreduce_sum = create_mapreduce_df(case=case_object, llm=llm, n=2)
mapreduce_sum.to_csv(output_folder / "mapreduce_final.csv")

# Refine Prompt Techniques

In [ ]:
from refine import create_refine_df

refine_sum = create_refine_df(case=case_object, llm=llm, n=2)
refine_sum.to_csv(output_folder / "refine_final.csv")